In [154]:
#importing necessary packages
import pandas as pd
import recordlinkage
import pathlib

# Loading Data 
The pre-processed bussniess record data is fed into the script using the load_data function. This data is then preprocessed further by ensuring correct data typing of the firm ID and the phone number and by creating a unique primary key called "ID".

In [155]:
#Read cleaned data into dataframe
Data_folder: str = pathlib.Path('C:\BBBdatagetsbetter\Data\cleaned_and_normalized_data_all.csv')
record_data = pd.read_csv(Data_folder)
record_data['firm_id'] = record_data['firm_id'].astype(str)
record_data['phone'] = record_data['phone'].astype(str)
record_data = record_data.reset_index(names=['ID'])
record_data

,ID,firm_id,company_name,address,city,state,zip,phone,email,url
0,0,2.0,"Able Fence, Inc.",78 Acker St E,Saint Paul,MN,55117.0,6512227737.0,afinc1008@qwestoffice.net,NaN
1,1,5.0,Albin Chapel,2200 Nicollet Ave,Minneapolis,MN,55404.0,nan,office@albinchapel.com,http://www.albinchapel.com/
2,2,5.0,Albin Funeral Chapel Inc,6855 Rowland Rd,Eden Prairie,MN,55344.0,6128711418.0,jimalbinson@gmail.com,NaN
3,3,5.0,"Albin Endeavor, Inc.",PO Box 46147,Eden Prairie,MN,55344.0,9529140043.0,NaN,NaN
4,4,7.0,Albrecht Company,1408 County Road C W,Roseville,MN,55113.0,6516334510.0,mail@albrechtcompany.com,https://www.albrechtcompany.com/
...,...,...,...,...,...,...,...,...,...,...
184721,184721,1000067470.0,Just Right Building Services,7283 Mallard Rd,Brook Park,MN,55007.0,nan,NaN,NaN
184722,184722,1000067471.0,Devils Lake Marine and Sports Center,1410 Kelly Dr NW,Devils Lake,ND,58301.0,7016621044.0,NaN,https://www.devilslakemarine.com/
184723,184723,1000067472.0,Beautiful Day Counseling Center LLC,300 3rd Ave SE Ste 408,Rochester,MN,55904.0,5072084150.0,caitlin@beautifuldaycounseling.com,https://www.beautifuldaycounseling.com/
184724,184724,1000067473.0,Everybody Healing Center LLC,8085 Wayzata Blvd #10,Golden Valley,MN,55426.0,6125672270.0,NaN,NaN


# Indexing Records
All possible duplicate record pairs are created using a blocking method which divides the records in such a way where the records within each group have a high likelihood of being duplicate records. These duplicate record pairs are known as candidate links.

In [156]:
#Setup indexer to make candiate pairs of records within the dataset. Block candiate pair generation on records that have
#valid condensed names. A data sample of 100000 is used for computing constraints. 
indexer = recordlinkage.Index()
data_sample = record_data
indexer.block("company_name")
indexer.block("address")
canidate_links = indexer.index(data_sample)
print(len(data_sample), len(canidate_links))

184726 116938


# Computing Similarity Score based on features

 Each candidate link is evaluated based on their simiarlity of company name, address, state, email, and url. These candidate links are then ranked to determine the best matches using a threshold

In [157]:
"""
Setup comparision methods used to determine potential records that match. Comparision methods can either be
Numeric, string-based, or exact matches. Specific comparrsion methods are a work in progress to determine which
features/methods are the best for generating accurate matches. 
"""
compare_cl = recordlinkage.Compare()
#compare_cl.string("condensed_name", "condensed_name",method ="levenshtein",threshold=0.75, label="condensed_name")
compare_cl.string(
    "company_name", "company_name", method="levenshtein", threshold=0.75, label="company_name"
)
compare_cl.string("address", "address", method="levenshtein", threshold=0.85, label="address_1")
compare_cl.exact("state", "state", label="state")
compare_cl.string(
    "email", "email", method="levenshtein", threshold=0.85, label="email"
)
compare_cl.string(
    "url", "url", label = "url"
)
# Computes the matching score of each record based on the candiate links generated and the comparision criteria. 
features = compare_cl.compute(canidate_links, data_sample)

As shown below, the similarity score of the matched record pair is computed for each record fair. The value represents a percentage match of each feature.

In [158]:
features.head(10)

,,company_name,address_1,state,email,url
12,11,0.0,1.0,1,0.0,0.787234
134,133,0.0,1.0,1,0.0,0.000000
135,131,0.0,1.0,1,0.0,0.000000
494,493,0.0,1.0,1,0.0,0.000000
778,708,1.0,0.0,0,0.0,0.000000
856,747,0.0,1.0,1,0.0,0.000000
1257,1036,0.0,1.0,1,0.0,0.000000
1297,1296,0.0,1.0,1,0.0,0.000000
1310,1309,1.0,1.0,1,0.0,0.000000
1349,27,1.0,1.0,1,0.0,0.000000


Below is a count of all the matched records obtained through the data indexing and feature comparrision process. Each pair has a total match value which shows how likely the record pair is a duplicate record.

In [159]:
#Finds the similarity score of overall records which is based on the sum of each feature being matched.
round(features.sum(axis=1)).value_counts().sort_index(ascending=False)

5.0       31
4.0    21467
3.0    15813
2.0    64365
1.0    15262
dtype: int64

In [160]:
#Potential matches are extracted from the features based on an arbirtray score of 3.5.
matches = features[round(features.sum(axis=1)) >= 4.5]
matches

,,company_name,address_1,state,email,url
31626,12001,1.0,1.0,1,1.0,1.000000
53160,53159,1.0,1.0,1,1.0,1.000000
59684,56086,1.0,1.0,1,1.0,0.939394
70393,22224,1.0,1.0,1,1.0,0.956522
75674,75152,1.0,1.0,1,1.0,1.000000
75700,75152,1.0,1.0,1,1.0,1.000000
82542,80097,1.0,1.0,1,1.0,0.965517
106751,86209,1.0,1.0,1,1.0,0.866667
110522,11005,1.0,1.0,1,1.0,0.571429
125281,125276,1.0,1.0,1,1.0,1.000000


Each pair is given a unique ID to be used for obtaining the firm id for each record

In [161]:
#The pairs of potential records are given IDs to be used for creating the final output
matches = matches.reset_index(names=['ID1','ID2'])
matches

,ID1,ID2,company_name,address_1,state,email,url
0,31626,12001,1.0,1.0,1,1.0,1.000000
1,53160,53159,1.0,1.0,1,1.0,1.000000
2,59684,56086,1.0,1.0,1,1.0,0.939394
3,70393,22224,1.0,1.0,1,1.0,0.956522
4,75674,75152,1.0,1.0,1,1.0,1.000000
5,75700,75152,1.0,1.0,1,1.0,1.000000
6,82542,80097,1.0,1.0,1,1.0,0.965517
7,106751,86209,1.0,1.0,1,1.0,0.866667
8,110522,11005,1.0,1.0,1,1.0,0.571429
9,125281,125276,1.0,1.0,1,1.0,1.000000


# Identifying Duplicate Records 
Using the unique "ID" of each paired record, the firm ID is extracted and placed into a list of tuples to identify 

In [162]:
def get_matches(data: pd.DataFrame, matches: pd.DataFrame) -> list:
    """
    Get matched 'firm_id' pairs.

    Args:
        data (pd.DataFrame): DataFrame containing the data.
        matches (pd.DataFrame): DataFrame containing matched record pairs.

    Returns:
        list: List of tuples containing matched 'firm_id' pairs.
    """
    def get_firm_id(id_list):
        temp = []
        for id in id_list:
            row = data[data['ID'] == id]
            firm_id = row.iloc[0]['firm_id']
            temp.append(firm_id)
        return temp
    
    id_list1 = matches.ID1.tolist()
    id_list2 = matches.ID2.tolist()
    list1 = get_firm_id(id_list1)
    list2 = get_firm_id(id_list2)
    matches_tuple = [(x, y) for x, y in zip(list1, list2)]
    return matches_tuple

In [163]:
results = get_matches(record_data,matches)
results

[('96016442.0', '21004636.0'),
 ('96099993.0', '96099992.0'),
 ('96123517.0', '96112007.0'),
 ('96146766.0', '96000350.0'),
 ('96159974.0', '96158896.0'),
 ('96160022.0', '96158896.0'),
 ('96204963.0', '96181169.0'),
 ('96502396.0', '96249102.0'),
 ('96540988.0', '20001775.0'),
 ('96569281.0', '96569277.0'),
 ('96572615.0', '96572613.0'),
 ('1000002172.0', '11003071.0'),
 ('1000010135.0', '1000010134.0'),
 ('1000010364.0', '96143934.0'),
 ('1000013103.0', '1000009127.0'),
 ('1000015515.0', '1000007231.0'),
 ('1000016258.0', '96369208.0'),
 ('1000027057.0', '96236263.0'),
 ('1000035178.0', '1000035176.0'),
 ('1000035191.0', '1000035190.0'),
 ('1000035193.0', '1000035192.0'),
 ('1000035799.0', '1000035798.0'),
 ('1000035940.0', '1000035933.0'),
 ('1000035996.0', '1000035888.0'),
 ('1000036007.0', '1000036005.0'),
 ('1000045842.0', '1000044797.0'),
 ('1000048532.0', '1000048388.0'),
 ('96114600.0', '96114504.0'),
 ('96114576.0', '96114546.0'),
 ('1000054087.0', '1000054086.0'),
 ('1000057

# Test Functions
Get 

In [98]:
#Returns the values of a specific column in a Dataframe
def getColumnValues(df, column_name: str) -> list[str]:
    return df[column_name]

In [124]:
#Retrieves records from a DataFrame based on the values in a specified ID column.
def getRecordsByID(df, ID_Column_header: str) -> list[str]:
    #Gets values from a specific column based on the column name
    column_values = getColumnValues(df, ID_Column_header)
    #Intializes a list based on all the column headers that were in the orginal record_data
    record_list = [record_data.columns.values.tolist()]
    #For every record in the column, find the corresponding record in the record_data and append its information
    #into the record_list
    for value in column_values:
        row = record_data[record_data['ID'] == value]
        record_list.append(row.values.tolist())
    return record_list


In [141]:
#Gets a list of record information from the record_data dataframe of the records that were in the matches dataframe
ID_list1 = getRecordsByID(matches,"ID1")
ID_list2 = getRecordsByID(matches,"ID2")

In [142]:
ID_list1

[['ID',
  'firm_id',
  'company_name',
  'address',
  'city',
  'state',
  'zip',
  'phone',
  'email',
  'url'],
 [[31626,
   '96016442.0',
   'Boyton Health Services',
   '410 Church St SE',
   'Minneapolis',
   'MN',
   55455.0,
   '6126258400.0',
   'canderson@bhs.umn.edu',
   'http://www.bhs.umn.edu']],
 [[53160,
   '96099993.0',
   'Red Wing Housing and Redevelopment Authority',
   '428 W 5th St',
   'Red Wing',
   'MN',
   55066.0,
   '6513850551.0',
   'randal.hemmerlin@ci.red-wing.mn.us',
   'http://www.redwinghra.org']],
 [[59684,
   '96123517.0',
   'Pinnacle Services Inc',
   '724 Central Ave NE',
   'Minneapolis',
   'MN',
   55414.0,
   '6129773100.0',
   'nicolas.thomley@pinnacleservices.org',
   'http://www.pinnacleservices.org']],
 [[70393,
   '96146766.0',
   'Talberg Lawn Landscape',
   '11770 Justen Cir',
   'Osseo',
   'MN',
   55369.0,
   '7634283550.0',
   'nfo@talberg.com',
   'http://www.talberg.com']],
 [[75674,
   '96159974.0',
   'Westside Liquor of Waite Pa

In [126]:
#Converts record data from a list of records and converts it into a Dataframe
def flattenRecords(record_list) -> pd.DataFrame:
    flattened_data = [item for sublist in record_list[1:] for item in sublist]
    df=pd.DataFrame(flattened_data, columns= ['ID', 'firm_id', 'company_name', 'address', 'city', 'state', 'zip',
                                             'phone', 'email', 'url'])
   # df.drop(['ID','company_name', 'condensed_name', 
   #         'main', 'legal', 'address_1', 'address_2', 'city', 
   #          'state', 'zip', 'zip_4', 'phone', 'email', 'url'],inplace=True,axis=1)
    return df

In [ ]:
def flattenRecords(record_list) -> pd.DataFrame:
    flattened_data = [item for sublist in record_list[1:] for item in sublist]
    df=pd.DataFrame(flattened_data, columns= ['firm_id'])
    df.drop(['condensed_name','main','legal','zip','zip_4'],inplace=True,axis=1)
    return df

In [104]:
#Uses the above function to convert the list of values from record_data using the IDs from the matches dataframe
df1 = flattenRecords(ID_list1)
df2 = flattenRecords(ID_list2)

In [105]:
df1

,ID,firm_id,company_name,address,city,state,zip,phone,email,url
0,31626,96016442.0,Boyton Health Services,410 Church St SE,Minneapolis,MN,55455.0,6126258400.0,canderson@bhs.umn.edu,http://www.bhs.umn.edu
1,53160,96099993.0,Red Wing Housing and Redevelopment Authority,428 W 5th St,Red Wing,MN,55066.0,6513850551.0,randal.hemmerlin@ci.red-wing.mn.us,http://www.redwinghra.org
2,59684,96123517.0,Pinnacle Services Inc,724 Central Ave NE,Minneapolis,MN,55414.0,6129773100.0,nicolas.thomley@pinnacleservices.org,http://www.pinnacleservices.org
3,70393,96146766.0,Talberg Lawn Landscape,11770 Justen Cir,Osseo,MN,55369.0,7634283550.0,nfo@talberg.com,http://www.talberg.com
4,75674,96159974.0,"Westside Liquor of Waite Park, Inc",8585 NE River Rd,Rice,MN,56367.0,3202539511.0,mholte@westsideliquor.com,https://facebook.com/westsideliquormn
5,75700,96160022.0,"Westside Liquor of Rice, Inc.",8585 NE River Rd,Rice,MN,56367.0,3203934513.0,mholte@westsideliquor.com,https://facebook.com/westsideliquormn
6,82542,96204963.0,"Homeopathic Medical Clinic, PA",4201 Excelsior Blvd,Minneapolis,MN,55416.0,9529338900.0,jmirman@demystify.com,http://www.bookonhealing.com
7,106751,96502396.0,"Arend Remodeling, LLC",1301 Cannon Ave,Mounds View,MN,55112.0,nan,mark.arend@usfamily.net,http://www.arendremodeling.com
8,110522,96540988.0,7O2 Communications,702 Main Ave,Moorhead,MN,56560.0,2182275166.0,702@702com.net,https://702com.net/
9,125281,96569281.0,Forman Market Incorporated,108 S Cascade St,Fergus Falls,MN,56537.0,2189984330.0,rickell@fintegra.net,http://www.richardell.com


In [106]:
df2

,ID,firm_id,company_name,address,city,state,zip,phone,email,url
0,12001,21004636.0,Boynton Health Services,410 Church St SE,Minneapolis,MN,55455.0,6126251434.0,canderson@bhs.umn.edu,http://www.bhs.umn.edu
1,53159,96099992.0,Red Wing Housing & Redevelopment Authority,428 W 5th St,Red Wing,MN,55066.0,6513887571.0,randal.hemmerlin@ci.red-wing.mn.us,http://www.redwinghra.org
2,56086,96112007.0,"Pinnacle Services, Inc",724 Central Ave NE,Minneapolis,MN,55414.0,6129773100.0,nicolas.thomley@pinnacleservices.org,https://www.pinnacleservices.org/
3,22224,96000350.0,Talberg Lawn & Landscaping,11770 Justen Cir,Maple Grove,MN,55369.0,7634283550.0,info@talberg.com,http://www.talberg.com/
4,75152,96158896.0,"Westside Liquor of Sartell, Inc.",8585 NE River Rd,Rice,MN,56367.0,3202408866.0,mholte@westsideliquor.com,https://facebook.com/westsideliquormn
5,75152,96158896.0,"Westside Liquor of Sartell, Inc.",8585 NE River Rd,Rice,MN,56367.0,3202408866.0,mholte@westsideliquor.com,https://facebook.com/westsideliquormn
6,80097,96181169.0,Homeopathic Medical Clinic PA,4201 Excelsior Blvd,St Louis Park,MN,55416.0,nan,jmirman@demystify.com,http://www.bookonhealing.com/
7,86209,96249102.0,Arend Remodeling,1301 Cannon Ave,Arden Hills,MN,55112.0,6513089756.0,mark.arend@usfamily.net,http://arendremodeling.com
8,11005,20001775.0,702 Communications,702 Main Ave,Moorhead,MN,56560.0,8777021118.0,702@702com.net,http://www.702com.com
9,125276,96569277.0,Milnor Market Incorporated,108 S Cascade St,Fergus Falls,MN,56537.0,2189984330.0,rickell@fintegra.net,http://www.richardell.com
